# Enable PII detection in test results

Learn how to enable Personally Identifiable Information (PII) detection when generating test results with the ValidMind Library, and configure safety overrides for specific test results.

## About ValidMind

ValidMind is a suite of tools for managing model risk, including risk associated with AI and statistical models. 

You use the ValidMind Library to automate documentation and validation tests, and then use the ValidMind Platform to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

### Before you begin

This notebook assumes you have basic familiarity with Python, including an understanding of how functions work. If you are new to Python, you can still run the notebook but we recommend further familiarizing yourself with the language. 

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

### New to ValidMind?

If you haven't already seen our documentation on the [ValidMind Library](https://docs.validmind.ai/developer/validmind-library.html), we recommend you begin by exploring the available resources in this section. There, you can learn more about documenting models and running tests, as well as find code samples and our Python Library API reference.

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>For access to all features available in this notebook, you'll need access to a ValidMind account.</b></span>
<br></br>
<a href="https://docs.validmind.ai/guide/configuration/register-with-validmind.html" style="color: #DE257E;"><b>Register with ValidMind</b></a></div>

### Key concepts

**Model documentation**: A structured and detailed record pertaining to a model, encompassing key components such as its underlying assumptions, methodologies, data sources, inputs, performance metrics, evaluations, limitations, and intended uses. It serves to ensure transparency, adherence to regulatory requirements, and a clear understanding of potential risks associated with the model’s application.

**Documentation template**: Functions as a test suite and lays out the structure of model documentation, segmented into various sections and sub-sections. Documentation templates define the structure of your model documentation, specifying the tests that should be run, and how the results should be displayed.

**Tests**: A function contained in the ValidMind Library, designed to run a specific quantitative test on the dataset or model. Tests are the building blocks of ValidMind, used to evaluate and document models and datasets, and can be run individually or as part of a suite defined by your model documentation template.

**Metrics**: A subset of tests that do not have thresholds. In the context of this notebook, metrics and tests can be thought of as interchangeable concepts.

**Custom metrics**: Custom metrics are functions that you define to evaluate your model or dataset. These functions can be registered with the ValidMind Library to be used in the ValidMind Platform.

**Inputs**: Objects to be evaluated and documented in the ValidMind Library. They can be any of the following:

  - **model**: A single model that has been initialized in ValidMind with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model).
  - **dataset**: Single dataset that has been initialized in ValidMind with [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset).
  - **models**: A list of ValidMind models - usually this is used when you want to compare multiple models in your custom metric.
  - **datasets**: A list of ValidMind datasets - usually this is used when you want to compare multiple datasets in your custom metric. (Learn more: [Run tests with multiple datasets](https://docs.validmind.ai/notebooks/how_to/run_tests_that_require_multiple_datasets.html))

**Parameters**: Additional arguments that can be passed when running a ValidMind test, used to pass additional information to a metric, customize its behavior, or provide additional context.

**Outputs**: Custom metrics can return elements like tables or plots. Tables may be a list of dictionaries (each representing a row) or a pandas DataFrame. Plots may be matplotlib or plotly figures.

**Test suites**: Collections of tests designed to run together to automate and generate model documentation end-to-end for specific use-cases.

Example: the [`classifier_full_suite`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html#ClassifierFullSuite) test suite runs tests from the [`tabular_dataset`](https://docs.validmind.ai/validmind/validmind/test_suites/tabular_datasets.html) and [`classifier`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html) test suites to fully document the data and model sections for binary classification model use-cases.

## Setting up

### Install the ValidMind Library with PII detection

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>Recommended Python versions</b></span>
<br></br>
Python 3.8 <= x <= 3.11</div>

To use PII detection powered by [Microsoft Presidio](https://microsoft.github.io/presidio/), install the library with the explicit `[pii-detection]` extra specifier:

In [ ]:
%pip install -q "validmind[pii-detection]"

### Initialize the ValidMind Library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the ValidMind Library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

#### Get your code snippet

1. In a browser, [log in to ValidMind](https://docs.validmind.ai/guide/configuration/log-in-to-validmind.html).

2. In the left sidebar, navigate to **Inventory** and click **+ Register Model**.

3. Enter the model details and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/model-inventory/register-models-in-inventory.html))

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, [load your model identifier credentials from an `.env` file](https://docs.validmind.ai/developer/model-documentation/store-credentials-in-env-file.html) or replace the placeholder with your own code snippet:

In [ ]:
# Load your model identifier credentials from an `.env` file

%load_ext dotenv
%dotenv .env

# Or replace with your code snippet

import validmind as vm

vm.init(
    # api_host="...",
    # api_key="...",
    # api_secret="...",
    # model="...",
)

## Using PII detection

### Create a custom test that outputs PII

To demonstrate the feature, we'll need a test that outputs PII. We'll create a custom test that returns:

- A description string containing PII (name, email, phone)
- A small table containing PII in columns

This output mirrors the structure used in other custom test notebooks and will exercise both table and description PII detection paths.

In [ ]:
import pandas as pd

from validmind import test

@test("my_pii_demo.PIIEmittingTest")
def pii_emitting_test():
    """A demo test that returns PII"""
    return pd.DataFrame(
        {
            "name": ["Jane Smith", "John Doe", "Alice Johnson"],
            "email": [
                "jane.smith@bank.example",
                "john.doe@company.example",
                "alice.johnson@service.example",
            ],
            "phone": ["(212) 555-9876", "(415) 555-1234", "(646) 555-5678"],
        }
    )

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>Want to learn more about custom tests?</b></span>
<br></br>
Check out our extended introduction to custom tests — <a href="https://docs.validmind.ai/notebooks/code_samples/custom_tests/implement_custom_tests.html" style="color: #DE257E;"><b>Implement custom tests</b></a></div>

### Run test under different PII detection modes

Now, let's import [the `run_test` function](https://docs.validmind.ai/validmind/validmind/tests.html#run_test) provided by the `validmind.tests` module to run our custom test.

We'll then switch `VALIDMIND_PII_DETECTION` across modes when we run our custom test and catch exceptions to observe blocking behavior.

In [ ]:
import os
from validmind.tests import run_test

#### disabled

No PII checks; test runs and logs:

In [ ]:
# Disabled Mode
print("\n=== Mode: disabled ===")
os.environ["VALIDMIND_PII_DETECTION"] = "disabled"
try:
    result = run_test("my_pii_demo.PIIEmittingTest")
    if not getattr(result, "_was_description_generated", False):
        print("Blocked: Test Description Generation was not run due to PII")
    else:
        print("Description was generated by LLM")
    try:
        result.log()
        print("Logging to API succeeded")
    except Exception as log_error:
        print("Blocked: Test Result was not logged due to PII")
        print("Reason:", log_error)
except Exception as e:
    print("Blocked: Test execution failed due to PII")
    print("Reason:", e)

#### test_results

Table PII triggers blocking; description may still proceed:

In [ ]:
# Test Results Mode
print("\n=== Mode: test_results ===")
os.environ["VALIDMIND_PII_DETECTION"] = "test_results"
try:
    result = run_test("my_pii_demo.PIIEmittingTest")
    if not getattr(result, "_was_description_generated", False):
        print("Blocked: Test Description Generation was not run due to PII")
    else:
        print("Description was generated by LLM")
    try:
        result.log()
        print("Logging to API succeeded")
    except Exception as log_error:
        print("Blocked: Test Result was not logged due to PII")
        print("Reason:", log_error)
except Exception as e:
    print("Blocked: Test execution failed due to PII")
    print("Reason:", e)

#### test_descriptions

Description PII triggers blocking; tables may still proceed:

In [ ]:
# Test Descriptions Mode
print("\n=== Mode: test_descriptions ===")
os.environ["VALIDMIND_PII_DETECTION"] = "test_descriptions"
try:
    result = run_test("my_pii_demo.PIIEmittingTest")
    if not getattr(result, "_was_description_generated", False):
        print("Blocked: Test Description Generation was not run due to PII")
    else:
        print("Description was generated by LLM")
    try:
        result.log()
        print("Logging to API succeeded")
    except Exception as log_error:
        print("Blocked: Test Result was not logged due to PII")
        print("Reason:", log_error)
except Exception as e:
    print("Blocked: Test execution failed due to PII")
    print("Reason:", e)

#### all

Both table and description checks are enforced:

In [ ]:
# All Mode
print("\n=== Mode: all ===")
os.environ["VALIDMIND_PII_DETECTION"] = "all"
try:
    result = run_test("my_pii_demo.PIIEmittingTest")
    if not getattr(result, "_was_description_generated", False):
        print("Blocked: Test Description Generation was not run due to PII")
    else:
        print("Description was generated by LLM")
    try:
        result.log()
        print("Logging to API succeeded")
    except Exception as log_error:
        print("Blocked: Test Result was not logged due to PII")
        print("Reason:", log_error)
except Exception as e:
    print("Blocked: Test execution failed due to PII")
    print("Reason:", e)

### Notes

- You can override blocking by passing `unsafe=True` to `result.log(unsafe=True)`, but this is not recommended outside controlled workflows.
- To test only a subset (tables or descriptions), adjust the test to emit only that type and re-run the mode loop.

### Troubleshooting

- If you see warnings that Presidio or Presidio analyzer is unavailable, ensure you installed extras: `validmind[pii-detection]`.
- If structured detection is unavailable, the library falls back to token-level text scans when possible.
- Ensure your environment is restarted after installing new packages if imports fail.

## Learn more

We offer many interactive notebooks to help you document models:

- [Run tests & test suites](https://docs.validmind.ai/guide/testing-overview.html)
- [Code samples](https://docs.validmind.ai/guide/samples-jupyter-notebooks.html)

Or, visit our [documentation](https://docs.validmind.ai/) to learn more about ValidMind.